In [ ]:
# Set tensorflow version to 1
%tensorflow_version 1.x

In [ ]:
# Imports
import tensorflow as tf
from google.colab import drive
import numpy as np
from tqdm.notebook import tqdm
import os
import shutil
import sys
import re
import json

drive.mount('/content/gdrive')
root_path = '/content/gdrive/MyDrive/Data/'
sys.path.append('/content/gdrive/MyDrive/shapestacks-master')
from tf_models.inception.inception_model import inception_v4_logregr_model_fn

In [ ]:
# extract files to local drive
shutil.unpack_archive(os.path.join(root_path, "shapestacks-rgb.tar.gz"), '/content/shapestacks-images')

In [ ]:
  # Configure runtime config and GPU options for forward pass
  gpu_options = tf.GPUOptions(
      allow_growth=True,
      per_process_gpu_memory_fraction=0.8
  )
  sess_config = tf.ConfigProto(gpu_options=gpu_options)
  run_config = tf.estimator.RunConfig(
      session_config=sess_config,
  )

In [ ]:
# Create our tf.Estimator predictor object
stability_predictor = tf.estimator.Estimator(
    model_fn=inception_v4_logregr_model_fn,
    # Downloaded pre-trained model from: https://ogroth.github.io/shapestacks/
    model_dir='/content/gdrive/MyDrive/Data/shapestacks-model/shapestacks-incpv4/shapestacks-cubes',
    config=run_config,
    params={})

In [ ]:
# Given an evironment name returns the perspective 
def get_perspective(filename):
  regex = re.compile(r'c=unique-cam_\d{1,2}')
  return int("".join(filter(lambda x: x.isdigit(), regex.search(filename).group(0))))

def is_stable(filename):
  vcom = re.compile(r'vcom=\d{1,2}')
  vpsf = re.compile(r'vpsf=\d{1,2}')

  # Extract numbers
  vcom = int("".join(filter(lambda x: x.isdigit(), vcom.search(filename).group(0))))
  vpsf = int("".join(filter(lambda x: x.isdigit(), vpsf.search(filename).group(0))))

  return not (vcom == 0 and vpsf == 0)

In [ ]:
def load_filenames():
  print("loading file names")
  root_path = '/content'
  base_dir = root_path +'/shapestacks-images/shapestacks/recordings'
  e_names = os.listdir(base_dir)
  environments = list(map(lambda x: os.path.join(base_dir, x), e_names))
  filenames = {}
  for i, e in tqdm(enumerate(environments)):
    images = [os.path.join(e, f) for f in os.listdir(e) if f.endswith('.png') and e.find('blocks') != -1]
    
    # Some folders don't have 16 images so we skip them if they don't
    if len(images) != 16:
      continue

    filenames[e_names[i]] = sorted(images, key=lambda x: get_perspective(x))

  return filenames

In [ ]:
def parse_record(filename, label):
  image_string = tf.read_file(filename)
  image_decoded = tf.image.decode_image(image_string, channels=3)
  image_resized = tf.image.resize_image_with_crop_or_pad(image_decoded, 224, 224)
  image_float = tf.cast(image_resized, tf.float32)
  image_float = tf.reshape(image_float, [224, 224, 3])
  return image_float, label

In [ ]:
def _create_dataset(filenames, labels):
  """
  Creates a dataset from the given filename and label tensors.
  """
  tf_filenames = tf.constant(filenames)
  tf_labels = tf.constant(labels)
  dataset = tf.data.Dataset.from_tensor_slices((tf_filenames, tf_labels))
  return dataset

In [ ]:
# Input function that acts as an iterator over the dataset
def input_fn(number_of_environments):
  files = load_filenames()
  if number_of_environments <= 0:
    filenames = np.array(list(files.values())).flatten()
  else:
    filenames = np.array(list(files.values())).flatten()[:number_of_environments * 16]

  dataset = _create_dataset(filenames, list(map(is_stable, filenames)))
  dataset = dataset.map(parse_record)
  dataset = dataset.prefetch(32)
  dataset = dataset.repeat(1)
  dataset = dataset.batch(8)

  iterator = dataset.make_one_shot_iterator()
  images, labels = iterator.get_next()
  return images, labels

In [ ]:
# Our target tensor generating function
def target_tensor(tensor, stability):
  return 1 - np.absolute(np.repeat(stability, 16) - tensor)

In [ ]:
NUM_OF_ENVIRONMENTS = 0  # Set to 0 to do all data on disk

# Runs the model on NUM_OF_ENVIRONMENTS environments
model_predictions = list(tqdm(stability_predictor.predict(lambda: input_fn(NUM_OF_ENVIRONMENTS))))

# Gets the probabities from the model predictions
probabilities = np.array([v['probabilities'][0] for v in model_predictions])

# Maps the output vectors to environment names
environments = list(load_filenames().keys())
outputs = { environments[i]: output_tensor for i, output_tensor in enumerate(np.array(np.split(probabilities, len(probabilities) // 16))) }

# Produce a mapping of environment name to target tensor
target_tensors = {}
for e, t in outputs.items():
  true_stability = is_stable(e)
  target_tensors[e] = list(map(float, target_tensor(t, true_stability)))[:12]

with open('/content/target_tensors_other.json', 'w+') as f:
    json.dump(target_tensors, f)
